# Install dependencies

```
pip install -r ../requirements.txt
pip install graphviz
apt-get update && apt-get install -y graphviz
```


# Code

In [ ]:
import parsimonious
import sys
import os
import re
import shlex

import graphviz

class DotParsimoniousVisitor(parsimonious.NodeVisitor):

    def __init__(self):
        self.count = 0
        self.dot = []

    def escape(self, str):
        return str.replace('"', '\\"')

    def generic_visit(self, node, visited_children):
        self.count += 1

        colors = {
            r'edit$': 'orange',
            r'typed_ref': 'skyblue',
            r'(.*_id|date|quoted)': 'pink',
            r'(code|alinea|article|paragraph|book|title|law|word|header\d+|.*lookback.*)_ref': 'cyan',
            r'.*_verb': 'green',
            r'.*_def': 'yellow'
        }

        if node.expr_name:
            c = 'white'
            for regex, color in colors.items():
                if re.match(regex, node.expr_name):
                    c = color
                    break
            self.dot += [
                '  %i [label="%s (%s)", style=filled, fillcolor=%s]'
                % (self.count, self.escape(node.text.strip()), node.expr_name, c)
            ]
        else:
            self.dot += [
                '  %i [label="%s (%s)", color=grey, fontcolor=grey]'
                % (self.count, self.escape(node.text.strip()), node.__class__.__name__)
            ]
        if len(visited_children):
            self.dot += ['  %i -- %s' % (self.count, ' -- '.join([str(i) for i in visited_children]))]

        return self.count

    def get_dot(self):
        return  'graph graphname {\nsize="18,18";\n%s\n}' % (';\n'.join(self.dot))

    
def print_syntax_tree(str):
    tree = grammar.match(str)

    v = DotParsimoniousVisitor()
    v.visit(tree)

    print(v.get_dot())

def render_syntax_tree(str):
    tree = grammar.match(str)

    v = DotParsimoniousVisitor()
    v.visit(tree)

    return graphviz.Source(v.get_dot())

# Grammar

In [ ]:
grammar = parsimonious.Grammar("""
alinea = (_* sentence)+ _*

sentence = (edit / ref_list / def) ~"[^.]*" "."?

edit =
    (edit_verb _ ref) /
    (ref _ edit_verb _ (preposition _)? (_ def)?) /
    (ref _ "est" _ (adverb _)? edit_verb (_ preposition)? (_ def)?) /
    (ref_list _ "sont" (_ adverb)? _ edit_verb (_ preposition)? (_ def)?)
edit_verb =
    ~"supprimer"i / 
    ~"supprimé(e|s|es)?" /
    ~"abrogé(e|s|es)?" /
    ~"modifié(e|s|es)?" /
    ~"rédigé(e|s|es)?" /
    ~"remplacer"i /
    ~"remplacé(e|s|es)?" /
    ~"ajouté(e|s|es)?" /
    ~"inséré(e|s|es)?" /
    ~"complété(e|s|es)?" /
    ~"ratifié(e|s|es)?" /
    ~"rétabli(e|s|es)?"



ref_list =
    (typed_ref (op_comma _ ref)* (_ op_and _ (ref_list / untyped_lookback_ref))+ (_ "de" _ typed_ref)?) /
    typed_ref

ref = (typed_ref / untyped_lookback_ref)
typed_ref = (positional_conjunction _)? (pronoun _?)? (law_ref / code_ref / book_ref / title_ref / paragraph_ref / article_ref / alinea_ref / word_ref / header1_ref) (((_ "de") / op_comma) _ typed_ref)?
untyped_lookback_ref = untyped_lookback_ref_id
untyped_lookback_ref_id = ordinal_adjective_number / number

alinea_ref =
    (~"alinéa(s)?"i _+ alinea_id) /
    (ordinal_adjective_number _ ~"alinéa(s)?"i) /
    lookback_alinea_ref
lookback_alinea_ref = ~"même(s)?"i _ ~"alinéa(s)?"i
alinea_id = number ""

article_ref = (~"article(s)?"i _+ article_id) / lookback_article_ref
lookback_article_ref = ~"même(s)?"i _ ~"article(s)?"i
article_id = number / ordinal_adjective_number

paragraph_ref =
    ((ordinal_adjective_number _)? ~"paragraphe(s)?"i) /
    (~"paragraphe(s)?"i _+ paragraph_id) /
    lookback_paragraph_ref
lookback_paragraph_ref = ~"même(s)?"i _ ~"paragraphe(s)?"i
paragraph_id = number ""

book_ref = (~"livre(s)"i _ book_id) / lookback_book_ref
lookback_book_ref = ~"même(s)?"i _ ~"livre(s)?"i
book_id = roman_number ""

title_ref = ~"titre(s)?"i _ title_id
title_id = roman_number ""

law_ref =
    (law_type _ ((number_abvr _* law_id _ "du" _ date) / (number_abvr _* law_id) / ("du" _ date))) /
    lookback_law_ref
lookback_law_ref = ~"même(s)?"i _ law_type
law_type = ~"loi( +constitutionnelle| +organique)?|ordonnance|d[ée]cret(-loi)?|arr[êe]t[ée]|circulaire"i
law_id = ~"[0-9]+[-‑][0-9]+"

word_ref = (positional_conjunction _)? (pronoun _)? (ordinal_adjective_number _)? word_type not_double_quote+ quoted

code_ref = code / lookback_code_ref
code = ( ~"code"i _ code_name ) / ~"constitution"i
lookback_code_ref = "même" _ "code"
code_name = ~"de +la +consommation +des +boissons +et +des +mesures +contre +l['’] *alcoolisme +applicable +dans +la +collectivité +territoriale +de +Mayotte|du +domaine +de +l['’] *Etat +et +des +collectivités +publiques +applicable +à +la +collectivité +territoriale +de +Mayotte|des +pensions +de +retraite +des +marins +français +du +commerce, +de +pêche +ou +de +plaisance|des +pensions +militaires +d['’] *invalidité +et +des +victimes +de +la +guerre|des +tribunaux +administratifs +et +des +cours +administratives +d['’] *appel|des +pensions +militaires +d['’] *invalidité +et +des +victimes +de +guerre|de +déontologie +des +professionnels +de +l['’] *expertise +comptable|de +déontologie +de +la +profession +de +commissaire +aux +comptes|de +l['’] *entrée +et +du +séjour +des +étrangers +et +du +droit +d['’] *asile|des +débits +de +boissons +et +des +mesures +contre +l['’] *alcoolisme|du +domaine +public +fluvial +et +de +la +navigation +intérieure|de +la +Légion +d['’] *honneur +et +de +la +médaille +militaire|des +relations +entre +le +public +et +l['’] *administration|de +l['’] *expropriation +pour +cause +d['’] *utilité +publique|général +de +la +propriété +des +personnes +publiques|des +postes +et +des +communications +électroniques|des +pensions +civiles +et +militaires +de +retraite|de +l['’] *Office +national +interprofessionnel +du +blé|de +déontologie +des +agents +de +police +municipale|disciplinaire +et +pénal +de +la +marine +marchande|des +instruments +monétaires +et +des +médailles|de +déontologie +des +chirurgiens-dentistes|général +des +collectivités +territoriales|de +la +construction +et +de +l['’] *habitation|de +déontologie +de +la +police +nationale|des +communes +de +la +Nouvelle-Calédonie|général +des +impôts, +annexe +2, +CGIAN2|général +des +impôts, +annexe +3, +CGIAN3|général +des +impôts, +annexe +4, +CGIAN4|général +des +impôts +annexe +1, +CGIAN1|de +l['’] *action +sociale +et +des +familles|des +procédures +civiles +d['’] *exécution|de +la +famille +et +de +l['’] *aide +sociale|de +l['’] *industrie +cinématographique|du +travail +applicable +à +Mayotte|de +déontologie +des +sages-femmes|de +déontologie +des +architectes|de +la +propriété +intellectuelle|du +cinéma +et +de +l['’] *image +animée|rural +et +de +la +pêche +maritime|de +l['’] *organisation +judiciaire|des +juridictions +financières|de +déontologie +des +médecins|de +l['’] *enseignement +technique|de +la +nationalité +française|de +déontologie +vétérinaire|de +justice +administrative|de +la +sécurité +intérieure|de +déontologie +médicale|général +des +impôts(, +CGI)?|de +la +sécurité +sociale|monétaire +et +financier|des +caisses +d['’] *épargne|de +la +voirie +routière|de +l['’] *aviation +civile|de +justice +militaire|de +la +santé +publique|du +domaine +de +l['’] *Etat|des +marchés +publics( +\(édition +(1964|2001|2004|2006)\))?|de +procédure +civile( +\(1807\))?|de +procédure +pénale|du +travail +maritime|du +service +national|des +ports +maritimes|de +l['’] *environnement|de +la +consommation|de +la +mutualité|de +la +recherche|de +l['’] *artisanat|de +l['’] *éducation|de +l['’] *urbanisme|des +assurances|des +transports|du +patrimoine|de +la +défense|des +communes|de +l['’] *énergie|des +douanes( +de +Mayotte)?|de +commerce|de +la +route|du +tourisme|du +travail|forestier( +de +Mayotte)?|électoral|du +sport|du +blé|du +vin|minier|pénal|rural|civil"i

header1_ref = header1_id ""
header1_id = roman_number (_ multiplicative_adverb)?



def = word_def / alinea_def

word_def =
    ((pronoun _?)? word_type (_ ~"suivant(s)?"i)? not_double_quote* quoted) /
    (_* ":" _ quoted)

alinea_def =
    ((pronoun _?)? ordinal_adjective_number _ "alinéa" (_ "ainsi" _ ~"rédigé(s)?" not_double_quote+ quoted)?) /
    (alinea_def_count _ ~"alinéa(s)?" (_ alinea_def_id)? (_ ~"suivant(s)?"i)? (_ "ainsi" _ ~"rédigé(s)?" not_double_quote+ quoted))
alinea_def_count = number / number_word
alinea_def_id = number ""



not_double_quote = ~"[^\\"]*"
quoted = "\\"" ~"[^\\n\\\"]+(\\n\\\"[^\\n\\\"]+)*" "\\""
word_type = ~"mot(s)?" / ~"nombre(s)?" / ~"chiffre(s)?" / "taux" / ~"référence(s)?" / ~"disposition(s)?"
date = day _ month _ year
day = ~"1er|[12][0-9]|3[01]|[1-9]"i
month = ~"janvier|février|mars|avril|mai|juin|juillet|août|septembre|octobre|novembre|décembre"i
year = ~"(1[5-9]|2[0-9])[0-9]{2}"
ordinal_adjective_number = ~"premi(er|ère)+|second(e)+|derni(er|ère)|dixième|onzième|douzième|treizième|quatorzième|quinzième|seizième|(dix-|vingt-|trente-|quarante-|cinquante-|soixante-|soixante-dix-|quatre-vingt-|quatre-vingt-dix-)?(et-)?(un|deux|trois|quatr|cinqu|six|sept|huit|neuv)ième"i
pronoun = ~"au"i / ~"de\w+l'"i / ~"de\w+la"i / ~"des"i / ~"les"i / ~"le"i / ~"la"i / ~"l'"i / ~"du"i / ~"un"i
roman_number = ~"Ier|[IVXLCDM]+(èm)?e?"
adverb = "ainsi"
preposition = "par"
positional_conjunction = ~"après"i / ~"avant"i / ~"au début"i / ~"à la fin"i
number = ~"\d+"
number_abvr = ~"n°|no"i
op_and = "et"
op_comma = ","
_ = ~"(\s+|\\n)"
number_word =
    "une" / "un" /
    "deux" /
    "second" /
    "trois" /
    "quatre" /
    "cinq" /
    "six" /
    "sept" /
    "huit" /
    "neuf" /
    "dix" /
    "onze" /
    "douze" /
    "treize" /
    "quatorze" /
    "quinze" /
    "seize"
multiplicative_adverb = ( multiplicative_adverb_units_before_decades? multiplicative_adverb_decades ) / multiplicative_adverb_units
multiplicative_adverb_units = ~"semel|bis|ter|quater|(quinqu|sex|sept|oct|no[nv])ies"i
multiplicative_adverb_units_before_decades = ~"un(de?)?|duo(de)?|ter|quater|quin|sex?|sept|octo|novo"i
multiplicative_adverb_decades = ~"(dec|v[ei]c|tr[ei]c|quadrag|quinquag|sexag|septuag|octog|nonag)ies"i
""")

# Tests

In [ ]:
render_syntax_tree("""
un premier alinéa
""")

In [ ]:
render_syntax_tree("""
la loi n°77-729 du 17 juillet 1977
""")

In [ ]:
render_syntax_tree("""
les mots "ceci est un test"
""")

In [ ]:
render_syntax_tree("""
après l'alinéa 4
""")

In [ ]:
render_syntax_tree("""
Au premier alinéa, les mots : ", par circonscription," sont supprimés.
""")

In [ ]:
render_syntax_tree("""
après l'alinéa 4 est inséré un alinéa 5 ainsi rédigé : "ceci est un test"
""")

In [ ]:
render_syntax_tree("""
les articles 3, 4 et 5 de la même loi sont remplacés par les mots "ceci est un test"
""")

In [ ]:
render_syntax_tree("""
Après l'alinéa 4 est inséré un cinquième alinéa ainsi rédigé : "ceci est un test".
Les articles 5 et 6 de la loi n°77-729 du 17 juillet 1977 sont supprimés.
Le I de l'article 4 de la loi n°88-42 est ainsi rédigé : "un autre test".
Les alinéas 2, 3 et 5 sont remplacés par un alinéa ainsi rédigé : "encore un test".
""")